In [63]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

channel =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv')
video =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv')
display(channel.head())
display(video.head())

,channelid,subcnt,viewcnt,videocnt,ct,channelname
0,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,410238653,736,2021-09-30 03:01:03,꽈뚜룹
1,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,736,2021-09-30 09:01:03,꽈뚜룹
2,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,735,2021-09-30 15:01:03,꽈뚜룹
3,UCkQCwnkQfgSuPTTnw_Y7v7w,1310000,412531322,737,2021-09-30 21:01:03,꽈뚜룹
4,UCkQCwnkQfgSuPTTnw_Y7v7w,1320000,412531322,737,2021-10-01 03:01:04,꽈뚜룹


,videopk,viewcnt,likecnt,dislikecnt,favoritecnt,cmcnt,ct,videoname
0,c5JQp6xafqc,1667010,30474,706,0,6587,2021-10-10 15:20:03,공범 EP1
1,c5JQp6xafqc,1669089,30495,707,0,6589,2021-10-10 15:30:03,공범 EP1
2,c5JQp6xafqc,1674759,30522,711,0,6596,2021-10-10 15:40:02,공범 EP1
3,c5JQp6xafqc,1677026,30555,712,0,6604,2021-10-10 15:50:03,공범 EP1
4,c5JQp6xafqc,1681824,30585,713,0,6600,2021-10-10 16:00:03,공범 EP1


In [6]:
## 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고 video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라
# 'ct' 컬럼을 datetype형식으로 변환
channel['ct'] = pd.to_datetime(channel['ct'])
video['ct'] = pd.to_datetime(video['ct'])

# videoname value count
print(video['videoname'].value_counts())

 공범 EP1    3492
 공범 EP2    3204
 공범 EP3    2568
 공범 EP4    2280
 공범 EP5    1562
 공범 EP6    1274
 공범 EP7     555
 공범 EP8     266
Name: videoname, dtype: int64


In [17]:
## 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라
print(video.groupby(['videoname']).max()[['ct', 'viewcnt']].reset_index())

  videoname                  ct  viewcnt
0    공범 EP1 2021-11-01 15:30:03  3180532
1    공범 EP2 2021-11-01 15:30:03  2199328
2    공범 EP3 2021-11-01 15:30:03  1671294
3    공범 EP4 2021-11-01 15:30:03  1818493
4    공범 EP5 2021-11-01 15:30:04  1503435
5    공범 EP6 2021-11-01 15:30:04  1750222
6    공범 EP7 2021-11-01 15:30:05  1630200
7    공범 EP8 2021-11-01 15:30:05  1289088


In [37]:
## Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라
channel1 = channel[channel['ct'] > '2021-10-03']
result = channel1.groupby(['channelid']).min()[['channelname', 'subcnt']].reset_index()
print(result.drop(columns='channelid'))

        channelname   subcnt
0            논리왕 전기   907000
1             야전삽짱재   257000
2           형사!탐정되다    14900
3       릴펄 Lilpearl    10100
4               조나단   215000
5        김농밀의 농밀한 삶     7520
6     Balming Tiger    54300
7               꽈뚜룹  1330000
8   와글와글 WagleWagle    54900
9        츄정ChuJeong   322000
10    곽토리 kwak tori   469000


In [35]:
## 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라
channel2 = channel[('2021-10-03 03:00:00' <= channel['ct']) & (channel['ct'] <= '2021-11-01 15:00:00')]

before = channel2.sort_values(by=['ct','channelname'], ascending=True).head(11).sort_values(by=['channelname'])
after = channel2.sort_values(by=['ct','channelname'], ascending=False).head(11).sort_values(by=['channelname'])
result = pd.DataFrame({'channelname':after['channelname'], 'del': [after['subcnt'].iloc[i] - before['subcnt'].iloc[i] for i in range(11)]})

display(result)

,channelname,del
1393,Balming Tiger,2500
1272,곽토리 kwak tori,-2000
1018,김농밀의 농밀한 삶,1540
129,꽈뚜룹,70000
510,논리왕 전기,-11000
1145,릴펄 Lilpearl,11000
256,야전삽짱재,11000
637,와글와글 WagleWagle,0
764,조나단,12000
383,츄정ChuJeong,1000


In [100]:
## 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다. 
## 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라
data1 = video[video['videoname'] == ' 공범 EP1'] # 공범 EP1만 추출
data1['ct'] = pd.to_datetime(data1['ct']) # 'ct'컬럼을 datetime으로 변경

data1['ct_lag1'] = data1['ct'].diff(1)
# data1['ct_del'] = data1['ct_lag1'] - data1['ct'] # 데이터 수집간격 계산
result = data1[(data1['ct_lag1']>='0 days 00:20:00') | (data1['ct_lag1']<='0 days 00:05:00')]['ct']
print(result)

10     2021-10-07 19:10:03
418    2021-10-10 17:00:03
721    2021-10-13 09:41:37
722    2021-10-13 09:41:37
1636   2021-10-19 18:20:03
Name: ct, dtype: datetime64[ns]


In [151]:
## 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라
video['ct'] = pd.to_datetime(video['ct'])
video['date'] = '.'
for i in range(len(video)):
    video['date'].iloc[i] = str(video['ct'].iloc[i].year) + '-' + str(video['ct'].iloc[i].month) + '-' + str(video['ct'].iloc[i].day)

video.date = pd.to_datetime(video.date)
result = video.groupby(['videoname']).min()['date']
print(result)

videoname
 공범 EP1   2021-10-07
 공범 EP2   2021-10-09
 공범 EP3   2021-10-14
 공범 EP4   2021-10-16
 공범 EP5   2021-10-21
 공범 EP6   2021-10-23
 공범 EP7   2021-10-28
 공범 EP8   2021-10-30
Name: date, dtype: datetime64[ns]


In [209]:
## “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다. 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라
video['hour'] = video['ct'].apply(lambda x: x.hour)
data1 = video[video['hour']==21].groupby('videoname').min().reset_index()
result= data1.sort_values(by='viewcnt', ascending=False)[['videoname', 'viewcnt', 'ct']]
print(result)

  videoname  viewcnt                  ct
7    공범 EP8   264029 2021-10-30 21:00:08
6    공범 EP7   252032 2021-10-28 21:00:04
3    공범 EP4   217674 2021-10-16 21:00:04
5    공범 EP6   213899 2021-10-23 21:00:06
4    공범 EP5   201179 2021-10-21 21:00:04
1    공범 EP2   148144 2021-10-09 21:00:03
2    공범 EP3   147183 2021-10-14 21:00:04
0    공범 EP1   117340 2021-10-07 21:00:03


In [208]:
## video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고 videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라
data1 = video.groupby('videoname').max().reset_index()
data1['ratio'] = data1['dislikecnt']/data1['likecnt']
data1[['videoname', 'ratio']].sort_values(by='ratio' ,ascending=True)

,videoname,ratio
6,공범 EP7,0.013636
5,공범 EP6,0.015881
3,공범 EP4,0.017971
4,공범 EP5,0.021128
2,공범 EP3,0.022359
0,공범 EP1,0.027118
1,공범 EP2,0.029844
7,공범 EP8,0.141357


In [239]:
## 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오
data1 = video[(video['ct'] >= '2021-11-01 00:00:00') & (video['ct'] <= '2021-11-01 15:00:00')]
result= pd.DataFrame(data1.groupby('videoname').max()['viewcnt'] - data1.groupby('videoname').min()['viewcnt']).reset_index()
print(result)

  videoname  viewcnt
0    공범 EP1    13298
1    공범 EP2    10300
2    공범 EP3     9927
3    공범 EP4     9824
4    공범 EP5    10824
5    공범 EP6    14141
6    공범 EP7    26949
7    공범 EP8    89147


In [222]:
## video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라
video[video.duplicated()][['videoname', 'ct']]

,videoname,ct
722,공범 EP1,2021-10-13 09:41:37
3927,공범 EP2,2021-10-13 09:41:37
